In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/conflict-data/deaths-in-armed news v2.csv


In [2]:
#Import data
df = pd.read_csv("/kaggle/input/conflict-data/deaths-in-armed news v2.csv")
#create db
conn = sqlite3.connect("database_name.db")
cursor = conn.cursor()

#Write the DataFrame to the SQL table
conflict_db = "Conflict_Data"
df.to_sql(conflict_db, conn, if_exists="replace", index=False)
#outputs
query = f"SELECT * FROM {conflict_db}"
result = pd.read_sql(query, conn)
print(result)

     Entity  Year  One-sided violence  Non-state  Intrastate  Extrasystemic  \
0    Africa  1989                3598       2507       34907              0   
1    Africa  1990                6574       1373       64839              0   
2    Africa  1991                5754       2707       25480              0   
3    Africa  1992                4087       7460        6887              0   
4    Africa  1993               11851      13985       15064              0   
..      ...   ...                 ...        ...         ...            ...   
205   World  2019                5536      20444       52173              0   
206   World  2020                8880      22401       72509              0   
207   World  2021               11935      23663      199568              0   
208   World  2022               11941      21430      185007              0   
209   World  2023               10275      20900       51662              0   

     Interstate  
0             0  
1             0

In [3]:
cursor.execute("""
ALTER TABLE Conflict_Data
ADD COLUMN total_violence INTEGER;
""")

In [4]:
cursor.execute("""
UPDATE Conflict_Data
SET total_violence = COALESCE("One-sided Violence", 0) 
                     + COALESCE("Non-state", 0)
                     + COALESCE("Intrastate", 0) 
                     + COALESCE("Extrasystemic", 0) 
                     + COALESCE("Interstate", 0);
""")
conn.commit()

In [5]:
#1. How many records are there in the table? 
query = f"SELECT COUNT(*) FROM {conflict_db}"
result = pd.read_sql(query, conn)
print(result)

   COUNT(*)
0       210


In [6]:
#2. What unique entity values are there?
query = f"SELECT DISTINCT Entity FROM {conflict_db}"
result = pd.read_sql(query, conn)
print(result)

             Entity
0            Africa
1          Americas
2  Asia and Oceania
3            Europe
4       Middle East
5             World


In [7]:
#3. How many records are there where Entity is Africa?
query = f"SELECT COUNT(*) FROM {conflict_db} WHERE Entity = 'Africa'"
result = pd.read_sql(query, conn)
print(result)

   COUNT(*)
0        35


In [8]:
#4. How many years are recorded per entity?
query = f"SELECT Entity, count(Year) as Incidents FROM {conflict_db} Group By Entity"
result = pd.read_sql(query, conn)
print(result)

             Entity  Incidents
0            Africa         35
1          Americas         35
2  Asia and Oceania         35
3            Europe         35
4       Middle East         35
5             World         35


In [9]:
#5. What are the values for One-sided Violence per entities since the 2000's?
query = f"""SELECT Entity, SUM("One-sided Violence") as Total
FROM Conflict_Data
WHERE Year >= 2000
GROUP BY Entity
ORDER BY Total ASC;"""
result = pd.read_sql(query, conn)
print(result)

             Entity   Total
0            Europe    4151
1          Americas   15410
2  Asia and Oceania   30474
3       Middle East   38218
4            Africa  117313
5             World  205566


In [10]:
#6. What about the total violence overall in this database?
query = f"""SELECT Entity, SUM("total_violence") as Total
FROM Conflict_Data
GROUP BY Entity
ORDER BY Total ASC;"""
result = pd.read_sql(query, conn)
print(result)

             Entity    Total
0          Americas   221763
1            Europe   291534
2  Asia and Oceania   578005
3       Middle East   682805
4            Africa  1991960
5             World  3766067


In [11]:
#7. What was the most violent year for each region?
query = f"""SELECT Entity, Year, MAX("total_violence") as Max_Incidents
FROM Conflict_Data
Group By Entity
ORDER BY  Max_Incidents ASC;"""
result = pd.read_sql(query, conn)
print(result)

             Entity  Year  Max_Incidents
0          Americas  2021          20088
1  Asia and Oceania  2021          39461
2            Europe  2022          93190
3       Middle East  2014         100153
4            Africa  1994         791652
5             World  1994         823473


In [12]:
#7. What drives violence in the Americas in 2021?
query = f"""SELECT *
FROM Conflict_Data
WHERE Entity = 'Americas' AND Year = 2021
Group By Entity;"""
result = pd.read_sql(query, conn)
print(result)

     Entity  Year  One-sided violence  Non-state  Intrastate  Extrasystemic  \
0  Americas  2021                 283      19692         113              0   

   Interstate  total_violence  
0           0           20088  
